# Final Project: Newspaper Headline/Landing Page Predicting Stock Market data

Layout:

1. Wayback machine and look into API's......see if any of the API's return the json. If I can't webscraping beutiful soup selnium....use selenium to click on a day and then the time. Maybe see what the top company.....then predict that companies stock.      
2. Do the headlines impact todays or tomorrows price.....index will be the stock price.  
3. NLP....tokenize. Remove stop words
4. Modeling: LSTM neural network or boosting/ensemble methods.....maybe model stacking 
  
Notes:  
- Data Sources: Stock data-Yahoo Finance, & Newspaper data-waybackmachine
- Do not use a CNN for this problem 
- Check out LSTM Neural Network 
- Pretrained word embedding 
- Avoid doing a day at a time do more batch
- Try maybe one newspaper at a time and try a different model for each
- Wayback machine for data



In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
import nltk
from nltk.tokenize import word_tokenize
np.random.seed(0)
import nltk
from nltk.corpus import gutenberg, stopwords
from nltk.collocations import *
from nltk import FreqDist
from nltk import word_tokenize
import string

## Retrieve Data

First we need to retrieve the financial market data from yahoo finance. Historical data in the form of a csv is available through direct download at the following link, https://finance.yahoo.com/quote/%5EGSPC/history?p=%5EGSPC.

In [2]:
sp_500=pd.read_csv('S&P 500 Data.csv')

In [3]:
sp_500.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,10/22/2018,2773.939941,2778.939941,2749.219971,2755.879883,2755.879883,3307140000
1,10/23/2018,2721.030029,2753.590088,2691.429932,2740.689941,2740.689941,4348580000
2,10/24/2018,2737.870117,2742.590088,2651.889893,2656.100098,2656.100098,4709310000
3,10/25/2018,2674.879883,2722.699951,2667.840088,2705.570068,2705.570068,4634770000
4,10/26/2018,2667.860107,2692.379883,2628.159912,2658.689941,2658.689941,4803150000


In [4]:
sp_500['Change']=sp_500['Open']-sp_500['Close']

In [5]:
sp_500_final= sp_500[['Date','Change']]

In [6]:
sp_500_final.head()

,Date,Change
0,10/22/2018,18.060058
1,10/23/2018,-19.659912
2,10/24/2018,81.770019
3,10/25/2018,-30.690185
4,10/26/2018,9.170166


In [7]:
Encode=pd.DataFrame(sp_500_final['Change'].where(sp_500_final['Change']<0, other=1))

In [99]:
encoded_data=pd.DataFrame(Encode.where(Encode>0,-1))

In [107]:
encoded_data.columns=['Coded']

In [108]:
sp_500=pd.concat([sp_500_final,encoded_data], axis=1)

In [109]:
sp_500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 3 columns):
Date      251 non-null object
Change    251 non-null float64
Coded     251 non-null float64
dtypes: float64(2), object(1)
memory usage: 6.0+ KB


In [110]:
sp_500.Date = pd.to_datetime(sp_500.Date)

In [111]:
sp_500.head()

,Date,Change,Coded
0,2018-10-22,18.060058,1.0
1,2018-10-23,-19.659912,-1.0
2,2018-10-24,81.770019,1.0
3,2018-10-25,-30.690185,-1.0
4,2018-10-26,9.170166,1.0


Now to use Beautful Soup to webscrape the wayback machine. The wayback machine is a open source website that stores historical webcrawls of many websites. I am going to use the wayback machine to webscrape the WSJ front page. From there I will scrape just the headlines and the sub headings to perform Natural Language Processing. 

In [20]:
url= 'https://web.archive.org/web/*/wsj.com'
driver = webdriver.Chrome(executable_path=r'C:\Users\GBLS\Downloads\chromedriver_win32\chromedriver')
driver.get('https://web.archive.org/web/*/wsj.com')

# python_button= driver.find_element_by_id


In [21]:
# python_button = driver.find_element_by_id('s3xx snapshot-link"')
# python_button.click()
# driver.implicitly_wait(30)
# continue_link = driver.find_element_by_link_text('1')
# continue_link.click()

In [22]:
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [23]:
print(soup.prettify())

<html lang="en">
 <head>
  <title>
   Wayback Machine
  </title>
  <script src="//archive.org/includes/jquery-1.10.2.min.js?v1.10.2" type="text/javascript">
  </script>
  <script src="//archive.org/includes/analytics.js?v=4983d3b9" type="text/javascript">
  </script>
  <script src="//archive.org/includes/build/npm/jquery-ui.min.js?v1.12.1" type="text/javascript">
  </script>
  <script src="//archive.org/includes/bootstrap.min.js?v3.0.0" type="text/javascript">
  </script>
  <script src="//archive.org/components/npm/clipboard/dist/clipboard.js?v=4983d3b9" type="text/javascript">
  </script>
  <script src="//archive.org/includes/build/npm/react/umd/react.production.min.js?v16.7.0" type="text/javascript">
  </script>
  <script src="//archive.org/includes/build/npm/react-dom/umd/react-dom.production.min.js?v16.7.0" type="text/javascript">
  </script>
  <script src="//archive.org/includes/build/js/archive.min.js?v=4983d3b9" type="text/javascript">
  </script>
  <script src="//archive.org/in

In [24]:
links=soup.find_all('a')

In [25]:
links

[<a data-action="ia-banner-close" href="#"></a>,
 <a class="navia-link home" href="/" target="_top" title="Home">
 <span class="iconochive-logo"></span>
 <span>Home</span>
 </a>,
 <a class="navia-link web" data-top-kind="web" href="https://archive.org/web/" target="_top" title="Web"><span aria-hidden="true" class="iconochive-web"></span><span>web</span></a>,
 <a class="navia-link texts" data-top-kind="texts" href="https://archive.org/details/texts" target="_top" title="Texts"><span aria-hidden="true" class="iconochive-texts"></span><span>books</span></a>,
 <a class="navia-link movies" data-top-kind="movies" href="https://archive.org/details/movies" target="_top" title="Video"><span aria-hidden="true" class="iconochive-movies"></span><span>video</span></a>,
 <a class="navia-link audio" data-top-kind="audio" href="https://archive.org/details/audio" target="_top" title="Audio"><span aria-hidden="true" class="iconochive-audio"></span><span>audio</span></a>,
 <a class="navia-link software" 

In [26]:
links_wsj= []
for x in links:
    if 'wsj' in x['href']:
        links_wsj.append(x['href'])

In [27]:
links_wsj

['/web/collections/*/wsj.com',
 '/web/changes/wsj.com',
 '/details/wsj.com',
 '/web/sitemap/wsj.com',
 '/web/19961203131021/wsj.com',
 '/web/20191024093558/wsj.com',
 '/web/20190101/wsj.com',
 '/web/20190102/wsj.com',
 '/web/20190103/wsj.com',
 '/web/20190104/wsj.com',
 '/web/20190105/wsj.com',
 '/web/20190106/wsj.com',
 '/web/20190107/wsj.com',
 '/web/20190108/wsj.com',
 '/web/20190109/wsj.com',
 '/web/20190110/wsj.com',
 '/web/20190111/wsj.com',
 '/web/20190112/wsj.com',
 '/web/20190113/wsj.com',
 '/web/20190114/wsj.com',
 '/web/20190115/wsj.com',
 '/web/20190116/wsj.com',
 '/web/20190117/wsj.com',
 '/web/20190118/wsj.com',
 '/web/20190119/wsj.com',
 '/web/20190120/wsj.com',
 '/web/20190121/wsj.com',
 '/web/20190122/wsj.com',
 '/web/20190123/wsj.com',
 '/web/20190124/wsj.com',
 '/web/20190125/wsj.com',
 '/web/20190126/wsj.com',
 '/web/20190127/wsj.com',
 '/web/20190128/wsj.com',
 '/web/20190129/wsj.com',
 '/web/20190130/wsj.com',
 '/web/20190131/wsj.com',
 '/web/20190201/wsj.com',
 '

In [28]:
full_link= []
for x in links_wsj:
    full_link.append('https://web.archive.org/'+ x)

In [29]:
full_link

['https://web.archive.org//web/collections/*/wsj.com',
 'https://web.archive.org//web/changes/wsj.com',
 'https://web.archive.org//details/wsj.com',
 'https://web.archive.org//web/sitemap/wsj.com',
 'https://web.archive.org//web/19961203131021/wsj.com',
 'https://web.archive.org//web/20191024093558/wsj.com',
 'https://web.archive.org//web/20190101/wsj.com',
 'https://web.archive.org//web/20190102/wsj.com',
 'https://web.archive.org//web/20190103/wsj.com',
 'https://web.archive.org//web/20190104/wsj.com',
 'https://web.archive.org//web/20190105/wsj.com',
 'https://web.archive.org//web/20190106/wsj.com',
 'https://web.archive.org//web/20190107/wsj.com',
 'https://web.archive.org//web/20190108/wsj.com',
 'https://web.archive.org//web/20190109/wsj.com',
 'https://web.archive.org//web/20190110/wsj.com',
 'https://web.archive.org//web/20190111/wsj.com',
 'https://web.archive.org//web/20190112/wsj.com',
 'https://web.archive.org//web/20190113/wsj.com',
 'https://web.archive.org//web/20190114/

In [86]:
january=full_link[6:37]

In [87]:
january

['https://web.archive.org//web/20190101/wsj.com',
 'https://web.archive.org//web/20190102/wsj.com',
 'https://web.archive.org//web/20190103/wsj.com',
 'https://web.archive.org//web/20190104/wsj.com',
 'https://web.archive.org//web/20190105/wsj.com',
 'https://web.archive.org//web/20190106/wsj.com',
 'https://web.archive.org//web/20190107/wsj.com',
 'https://web.archive.org//web/20190108/wsj.com',
 'https://web.archive.org//web/20190109/wsj.com',
 'https://web.archive.org//web/20190110/wsj.com',
 'https://web.archive.org//web/20190111/wsj.com',
 'https://web.archive.org//web/20190112/wsj.com',
 'https://web.archive.org//web/20190113/wsj.com',
 'https://web.archive.org//web/20190114/wsj.com',
 'https://web.archive.org//web/20190115/wsj.com',
 'https://web.archive.org//web/20190116/wsj.com',
 'https://web.archive.org//web/20190117/wsj.com',
 'https://web.archive.org//web/20190118/wsj.com',
 'https://web.archive.org//web/20190119/wsj.com',
 'https://web.archive.org//web/20190120/wsj.com',


In [88]:
# driver.get(str(january[0]))
# my_html= driver.page_source
# soup= BeautifulSoup(my_html, 'html.parser')
# for div in soup.findAll('a', {'class': 'wsj-headline-link'}):
#     print(div.contents[0])

In [89]:
date=[]
headline_blurb_data= []
for link in january: #FULLY FUNCTIONAL SCRAPE OF HEADLINES WHEN PASS IN LIST OF LINKS
    driver.get(str(link))
    my_html= driver.page_source
    soup= BeautifulSoup(my_html, 'html.parser')
    for div in soup.findAll('a', {'class': 'wsj-headline-link'}):
        headline_blurb_data.append(div.contents[0])
        date.append(str(link)[29:37])
    for div in soup.findAll('p', {'class':'wsj-summary'}): #Getting all the headlines
        headline_blurb_data.append(div.text)
        date.append(str(link)[29:37])
df_wsj=pd.DataFrame({'Date':date,'Text':headline_blurb_data})
df_wsj.Date= pd.to_datetime(df_wsj.Date)
df_final=df_wsj.merge(sp_500,how='left')        
    

,Date,Text,Change,Change
0,2019-01-01,Trump Invites Top Lawmakers in Effort To End S...,NaN,NaN
1,2019-01-01,Kim Jong Un Extends Peace Overture to U.S.,NaN,NaN
2,2019-01-01,"American Detained in Russia Isn’t a Spy, Famil...",NaN,NaN
3,2019-01-01,The Money Managers to Watch in 2019,NaN,NaN
4,2019-01-01,Investors Try Not to Panic Over Stock Volatility,NaN,NaN
5,2019-01-01,Rewards Credit Cards Gained a Fanatic Followin...,NaN,NaN
6,2019-01-01,Chesapeake Energy Bet on Oil. Then Crude Price...,NaN,NaN
7,2019-01-01,Brazil’s Idea to Fix Rampant Gun Violence: Mor...,NaN,NaN
8,2019-01-01,"Conservative Takes Reins in Brazil, Vows to Re...",NaN,NaN
9,2019-01-01,"To Woo Millennials, Atlanta Weighs Parks Over ...",NaN,NaN


In [90]:
df_wsj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3747 entries, 0 to 3746
Data columns (total 2 columns):
Date    3747 non-null datetime64[ns]
Text    3747 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 58.6+ KB


In [146]:
final=df_wsj.merge(sp_500,'left')

In [149]:
final.to_csv("January Data")

In [ ]:
#COME BACK TO WRITE LOGIC OR GO ABOVE AT BEG TO WRITE LOGIC TO FILL IN NAN IT SHOULD TAKE THE DAY BEFORE ITS CLOSING PRICE

In [142]:
final=final.fillna(value=0, inplace=True)

In [147]:
final

,Date,Text,Change,Coded
0,2019-01-01,Trump Invites Top Lawmakers in Effort To End S...,NaN,NaN
1,2019-01-01,Kim Jong Un Extends Peace Overture to U.S.,NaN,NaN
2,2019-01-01,"American Detained in Russia Isn’t a Spy, Famil...",NaN,NaN
3,2019-01-01,The Money Managers to Watch in 2019,NaN,NaN
4,2019-01-01,Investors Try Not to Panic Over Stock Volatility,NaN,NaN
5,2019-01-01,Rewards Credit Cards Gained a Fanatic Followin...,NaN,NaN
6,2019-01-01,Chesapeake Energy Bet on Oil. Then Crude Price...,NaN,NaN
7,2019-01-01,Brazil’s Idea to Fix Rampant Gun Violence: Mor...,NaN,NaN
8,2019-01-01,"Conservative Takes Reins in Brazil, Vows to Re...",NaN,NaN
9,2019-01-01,"To Woo Millennials, Atlanta Weighs Parks Over ...",NaN,NaN


In [144]:
final

## Baseline Model

In [136]:
from sklearn.model_selection import train_test_split
X=final['Text']
y=final['Coded']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.25, random_state=1)

In [139]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer= CountVectorizer()
vectorizer.fit(X_train)


X_train
# X_train= vectorizer.transform(X_train)
# X_test= vectorizer.transform(X_test)



AttributeError: lower not found

### Sentiment Analysis

In [ ]:
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as plt
%matplotlib inline

### Getting Headline Blurbs

In [21]:
# headlines=[]
# for div in soup.findAll('a', {'class':'wsj-headline-link'}): #Getting all the headlines
#     headlines.append(div.contents[0])
#     print(div.contents[0])

Do we want to split up the intervals of stuff so we essentially would do say a month then predict what hte stock does over a month?

Do we keep it to a day and just do what the day is and then from there then see where it goes in a month 

How will we divide up the different months......for loop to go through the different 1-30.......way to stop after reaching 30 to create new lists

Do it on a weekly basis so take 7 days of data then predict the stock price for the next week?





In [61]:
driver.get('https://web.archive.org/web/20190102000911/https://www.wsj.com/')
my_html = driver.page_source
soup = BeautifulSoup(my_html, 'html.parser')

In [62]:
wsj_homepage=soup

In [65]:
wsj_homepage.findAll('a', {'class':'wsj-headline-link'})

[<a class="wsj-headline-link" data-reactid=".17szcl7ii7k.1.1.5.1.1.1.3.0.0.0.0.0:$0.1.0" href="https://web.archive.org/web/20190102000911/https://www.wsj.com/articles/white-house-invites-congressional-leaders-to-meet-with-trump-wednesday-11546374858?mod=hp_lead_pos1">Trump Invites Top Lawmakers in Effort To End Shutdown</a>,
 <a class="wsj-headline-link" data-reactid=".17szcl7ii7k.1.1.5.1.1.1.3.0.0.0.0.1:$1.1.0" href="https://web.archive.org/web/20190102000911/https://www.wsj.com/articles/kim-jong-un-hopes-for-new-summit-with-trump-but-issues-warning-11546311683?mod=hp_lead_pos2">Kim Jong Un Extends Peace Overture to U.S.</a>,
 <a class="wsj-headline-link" data-reactid=".17szcl7ii7k.1.1.5.1.1.1.3.0.0.0.0.1:$2.1.0" href="https://web.archive.org/web/20190102000911/https://www.wsj.com/articles/american-detained-in-russia-wasnt-spy-family-says-11546349660?mod=hp_lead_pos3">American Detained in Russia Isn’t a Spy, Family Says</a>,
 <a class="wsj-headline-link" data-reactid=".17szcl7ii7k.1.1

### Tokenization

In [18]:
# headlines=[]
# for div in wsj_homepage.findAll('a', {'class':'wsj-headline-link'}): #Getting all the headlines
#     headlines.append(div.contents[0])
#     print(div.contents[0])

In [12]:
# string_headlines= str(headlines)

In [36]:
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',"’", '‘'] 

In [37]:
tokens = nltk.word_tokenize(string_headlines)
stop_words_removed= [token.lower() for token in tokens if token not in stopwords_list] 

In [38]:
Freq_Dist_Headlines=FreqDist(stop_words_removed)

In [39]:
Freq_Dist_Headlines.most_common()

[('2018', 10),
 ('2019', 9),
 ("'the", 7),
 ('with', 7),
 ('a', 7),
 ('shutdown', 5),
 ('year', 5),
 ('new', 5),
 ('u.s.', 4),
 ('u.s', 4),
 ('best', 4),
 ('stocks', 4),
 ('companies', 4),
 ('says', 3),
 ("'investors", 3),
 ('over', 3),
 ('out', 3),
 ('way', 3),
 ('continues', 3),
 ('market', 3),
 ('is', 3),
 ('that', 3),
 ('be', 3),
 ('tech', 3),
 ('may', 3),
 ('million', 3),
 ('happened', 3),
 ("'trump", 2),
 ('end', 2),
 ('isn', 2),
 ('money', 2),
 ('volatility', 2),
 ('back', 2),
 ('more', 2),
 ('guns', 2),
 ('takes', 2),
 ("'to", 2),
 ('nfl', 2),
 ('pentagon', 2),
 ('risk', 2),
 ("'new", 2),
 ('talent', 2),
 ('stories', 2),
 ('after', 2),
 ("'great", 2),
 ('economy', 2),
 ("'amazon", 2),
 ('battle', 2),
 ('fall', 2),
 ("'what", 2),
 ('can', 2),
 ('workers', 2),
 ('rebound', 2),
 ('stay', 2),
 ('home', 2),
 ('auctions', 2),
 ('luxury', 2),
 ('real-estate', 2),
 ('latest', 2),
 ('alabama', 2),
 ('cios', 2),
 ('plan', 2),
 ('sales', 2),
 ("'every", 2),
 ('investor', 2),
 ('humbled', 

In [92]:
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

headlines_stopped= [word for word in tokens if word not in stopwords_list]

In [94]:
FreqDist(headlines_stopped).most_common()

[('’', 39),
 ('2018', 10),
 ('2019', 9),
 ('‘', 9),
 ("'the", 7),
 ('u.s.', 6),
 ('shutdown', 5),
 ('year', 5),
 ('new', 5),
 ('best', 4),
 ('stocks', 4),
 ('companies', 4),
 ('says', 3),
 ("'investors", 3),
 ('way', 3),
 ('continues', 3),
 ('market', 3),
 ('tech', 3),
 ('may', 3),
 ('million', 3),
 ('happened', 3),
 ("'trump", 2),
 ('end', 2),
 ('money', 2),
 ('volatility', 2),
 ('back', 2),
 ('guns', 2),
 ('takes', 2),
 ("'to", 2),
 ('nfl', 2),
 ('pentagon', 2),
 ('risk', 2),
 ("'new", 2),
 ('talent', 2),
 ('stories', 2),
 ("'great", 2),
 ('u.s', 2),
 ('economy', 2),
 ("'amazon", 2),
 ('battle', 2),
 ('fall', 2),
 ("'what", 2),
 ('workers', 2),
 ('rebound', 2),
 ('stay', 2),
 ('home', 2),
 ('auctions', 2),
 ('luxury', 2),
 ('real-estate', 2),
 ('latest', 2),
 ('alabama', 2),
 ('cios', 2),
 ('plan', 2),
 ('sales', 2),
 ("'every", 2),
 ('investor', 2),
 ('humbled', 2),
 ('wise', 2),
 ('figure', 2),
 ('invites', 1),
 ('top', 1),
 ('lawmakers', 1),
 ('effort', 1),
 ("'kim", 1),
 ('jong',

## Subheadings

In [67]:
#Getting all the subheading text

In [68]:
blurb[:5]

['President Trump ramped up pressure on Democrats to end an impasse over funding for a border wall, formally inviting them to a bipartisan meeting at the White House on Wednesday. 474',
 'North Korean leader Kim Jong Un offered a new peace overture in his New Year’s address, saying his country is refraining from producing nuclear weapons, a gesture some experts interpreted as a potential opening for resuming talks with President Trump. 50',
 'The family of an American detained in Russia over spying allegations said he was traveling in the country for a wedding and denied he was involved in espionage. 83',
 'Hedge-fund managers once again lagged the broader market and will have to show they are worth their high fees. Those that don’t deliver this year might have to make difficult decisions. These are four leaders to watch.',
 'For stock-market investors, times of heightened volatility can sometimes be a godsend—“as long as you can stomach the headlines.”']

In [58]:
tokens_blurbs= nltk.word_tokenize(str(blurb))

In [59]:
clean_tokens_blurbs=[token for token in tokens_blurbs if token not in stopwords_list]

In [60]:
Freq_Dist_blurbs=FreqDist(clean_tokens_blurbs)

In [61]:
Freq_Dist_blurbs.most_common()

[('new', 11),
 ("'The", 7),
 ('year', 7),
 ('Trump', 4),
 ('potential', 3),
 ('market', 3),
 ('military', 3),
 ('across', 3),
 ('U.S.', 3),
 ('2018', 3),
 ('even', 3),
 ('tech', 3),
 ('Democrats', 2),
 ('end', 2),
 ('New', 2),
 ('Year', 2),
 ('country', 2),
 ('President', 2),
 ('broader', 2),
 ('make', 2),
 ('investors', 2),
 ('like', 2),
 ('Now', 2),
 ('banks', 2),
 ('oil', 2),
 ('time', 2),
 ('since', 2),
 ('Department', 2),
 ('seasonal', 2),
 ('work', 2),
 ('A', 2),
 ('vowing', 2),
 ('financial', 2),
 ('top', 2),
 ('stories', 2),
 ('past', 2),
 ('much', 2),
 ('debt', 2),
 ('consumer', 2),
 ('far', 2),
 ('known', 2),
 ("'President", 1),
 ('ramped', 1),
 ('pressure', 1),
 ('impasse', 1),
 ('funding', 1),
 ('border', 1),
 ('wall', 1),
 ('formally', 1),
 ('inviting', 1),
 ('bipartisan', 1),
 ('meeting', 1),
 ('White', 1),
 ('House', 1),
 ('Wednesday', 1),
 ('474', 1),
 ("'North", 1),
 ('Korean', 1),
 ('leader', 1),
 ('Kim', 1),
 ('Jong', 1),
 ('Un', 1),
 ('offered', 1),
 ('peace', 1),
 

In [65]:
text_combined= blurb + headlines

In [68]:
tokenized_combined= nltk.word_tokenize(str(text_combined))

In [73]:
stop_wordless_combine= [token.lower() for token in tokenized_combined if token not in stopwords_list ]

In [74]:
FreqDist(stop_wordless_combine).most_common()

[('new', 18),
 ('year', 14),
 ("'the", 14),
 ('2018', 13),
 ('a', 9),
 ('2019', 9),
 ('u.s.', 7),
 ('with', 7),
 ('market', 6),
 ('tech', 6),
 ('trump', 5),
 ('best', 5),
 ('u.s', 5),
 ('shutdown', 5),
 ('end', 4),
 ('top', 4),
 ('stories', 4),
 ("'investors", 4),
 ('stocks', 4),
 ('companies', 4),
 ('democrats', 3),
 ('potential', 3),
 ('investors', 3),
 ('volatility', 3),
 ('back', 3),
 ('banks', 3),
 ('figure', 3),
 ('oil', 3),
 ('since', 3),
 ('pentagon', 3),
 ('military', 3),
 ('across', 3),
 ('chief', 3),
 ('financial', 3),
 ('past', 3),
 ('even', 3),
 ('debt', 3),
 ('consumer', 3),
 ('battle', 3),
 ('workers', 3),
 ("'new", 3),
 ('says', 3),
 ('over', 3),
 ('out', 3),
 ('way', 3),
 ('continues', 3),
 ('is', 3),
 ('that', 3),
 ('be', 3),
 ('may', 3),
 ('million', 3),
 ('happened', 3),
 ('wall', 2),
 ('white', 2),
 ('jong', 2),
 ('un', 2),
 ('peace', 2),
 ('overture', 2),
 ('country', 2),
 ('nuclear', 2),
 ('president', 2),
 ('family', 2),
 ('american', 2),
 ('detained', 2),
 ('ru

https://stackoverflow.com/questions/17870528/double-clicking-in-python-selenium